In [1]:
! pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d kaushiksuresh147/bitcoin-tweets

Dataset URL: https://www.kaggle.com/datasets/kaushiksuresh147/bitcoin-tweets
License(s): CC0-1.0
100% 695M/695M [00:25<00:00, 28.9MB/s]
100% 695M/695M [00:25<00:00, 28.5MB/s]


In [4]:
!kaggle datasets download -d aisolutions353/btc-tweets-sentiment

Dataset URL: https://www.kaggle.com/datasets/aisolutions353/btc-tweets-sentiment
License(s): unknown
100% 3.64M/3.64M [00:00<00:00, 5.45MB/s]
100% 3.64M/3.64M [00:00<00:00, 4.02MB/s]


In [5]:
from zipfile import ZipFile
dataset = '/content/btc-tweets-sentiment.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('done')

done


In [6]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib


In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [9]:
twitter_dataset = pd.read_csv('/content/BTC_Tweets_Updated.csv', encoding= 'ISO-8859-1')
tweets = twitter_dataset["Tweet"].astype(str)

In [10]:
twitter_dataset.shape

(50852, 11)

In [11]:
twitter_dataset.head()

,id,Date,Tweet,Screen_name,Source,Link,Sentiment,sent_score,New_Sentiment_Score,New_Sentiment_State,BERT Labels
0,0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['neutral'],0.0,0.000000,0.0,1
1,1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",['neutral'],0.0,0.000000,0.0,0
2,2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,hojachotopur,"[u'blockchain', u'Tipper', u'TipperEconomy']","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.136364,1.0,0
3,3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,denies_distro,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.400000,1.0,0
4,4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,aditzgraha,[],"<a href=""http://twitter.com/download/android"" ...",['positive'],1.0,0.468182,1.0,0


In [12]:
twitter_dataset.isnull().sum()

,0
id,0
Date,0
Tweet,0
Screen_name,0
Source,0
Link,0
Sentiment,0
sent_score,0
New_Sentiment_Score,0
New_Sentiment_State,0


In [13]:
twitter_dataset['Sentiment'].value_counts()

,count
Sentiment,
['positive'],22937
['neutral'],21932
['negative'],5983


In [14]:
from sklearn.utils import resample

In [15]:
# Update the filtering to match the exact sentiment labels
positive = twitter_dataset[twitter_dataset['Sentiment'] == "['positive']"]
neutral = twitter_dataset[twitter_dataset['Sentiment'] == "['neutral']"]
negative = twitter_dataset[twitter_dataset['Sentiment'] == "['negative']"]

# Verify that each subset DataFrame contains data
if len(positive) == 0 or len(neutral) == 0 or len(negative) == 0:
    raise ValueError("One of the sentiment categories is empty. Please check your dataset and labels.")

# Set target size to the maximum count across classes
target_size = max(len(positive), len(neutral), len(negative))

# Upsample the neutral and negative classes to match the target size
neutral_upsampled = resample(neutral, replace=True, n_samples=target_size, random_state=42)
negative_upsampled = resample(negative, replace=True, n_samples=target_size, random_state=42)

# Combine the balanced classes into a new DataFrame
balanced_dataset = pd.concat([positive, neutral_upsampled, negative_upsampled])

# Shuffle the dataset to mix classes
balanced_dataset = balanced_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Verify the new distribution
print(balanced_dataset['Sentiment'].value_counts())

Sentiment
['negative']    22937
['neutral']     22937
['positive']    22937
Name: count, dtype: int64


In [16]:
print("Unique sentiment values:", twitter_dataset['Sentiment'].unique())

Unique sentiment values: ["['neutral']" "['positive']" "['negative']"]


In [17]:
def stemming(content):
  stemmed_content = re.sub(r'[^\w\s]',' ',content)
  stemmed_content = re.sub(r'[\U0001F600-\U0001F64F]', ' ', content)
  stemmed_content = stemmed_content.lower()

  return stemmed_content

In [18]:
twitter_dataset['stemmed_tweets'] = twitter_dataset['Tweet'].apply(stemming)

In [19]:
twitter_dataset.head()

,id,Date,Tweet,Screen_name,Source,Link,Sentiment,sent_score,New_Sentiment_Score,New_Sentiment_State,BERT Labels,stemmed_tweets
0,0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['neutral'],0.0,0.000000,0.0,1,"rt @alxtoken: paul krugman, nobel luddite. i h..."
1,1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",['neutral'],0.0,0.000000,0.0,0,@lopp @_kevin_pham @psycho_sage @naval but @pr...
2,2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,hojachotopur,"[u'blockchain', u'Tipper', u'TipperEconomy']","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.136364,1.0,0,rt @tippereconomy: another use case for #block...
3,3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,denies_distro,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.400000,1.0,0,free coins https://t.co/diuoepjdap
4,4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,aditzgraha,[],"<a href=""http://twitter.com/download/android"" ...",['positive'],1.0,0.468182,1.0,0,rt @payvxofficial: we are happy to announce th...


In [20]:
x = twitter_dataset['stemmed_tweets'].values
y = twitter_dataset['New_Sentiment_Score'].values

In [21]:
print(x)

["rt @alxtoken: paul krugman, nobel luddite. i had to tweak the nose of this bitcoin enemy. he says such foolish things. here's the link: httâ\x80¦"
 '@lopp @_kevin_pham @psycho_sage @naval but @proffaustus (dum b a ss) said you know nothing about #bitcoin ... ð\x9f\x98\x82ð\x9f\x98\x82ð\x9f\x98\x82 https://t.co/sbamfq2yiy'
 'rt @tippereconomy: another use case for #blockchain and #tipper. the #tippereconomy  can unseat facebook and change everything! ico live noâ\x80¦'
 ...
 'rt @gymrewards: buy #gymrewards tokens, bonus time is ending! https://t.co/hdvhozrz2j, #ico #cryptocurrency #mobile #app #mining #exercisinâ\x80¦'
 'i added a video to a @youtube playlist https://t.co/ntfjrnvsvz how to bitcoin cloud mining free for lifetime urdu / hindi'
 'rt @raybambs: airdrop photocoin airdrop round#2. 100 #photocoin will be giving to everyone who complete the google form. your account willâ\x80¦']


In [22]:
print(y)

[ 0.          0.          0.13636364 ...  0.          0.4
 -0.05      ]


In [23]:
z  = twitter_dataset['Sentiment'].values

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=twitter_dataset['Sentiment'])

In [25]:
print(x.shape, x_train.shape, x_test.shape)

(50852,) (40681,) (10171,)


In [26]:
vectorizer = TfidfVectorizer()
vectorizer.fit(tweets)

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [27]:
print(x_train, x_test)

  (0, 36235)	0.5012828657600646
  (0, 22981)	0.18533226599145705
  (0, 16606)	0.18609140877232982
  (0, 47359)	0.14343642761058661
  (0, 20984)	0.19352766090053494
  (0, 29953)	0.19854568564403896
  (0, 20203)	0.1965502400958868
  (0, 10807)	0.0718964356886653
  (0, 23137)	0.13265726088648067
  (0, 13579)	0.13535881002048797
  (0, 15665)	0.5012828657600646
  (0, 45446)	0.5012828657600646
  (1, 37518)	0.06953218542624932
  (1, 10532)	0.153159181130839
  (1, 44989)	0.1427875267740436
  (1, 22278)	0.16958297052368185
  (1, 25901)	0.20071906140007267
  (1, 22661)	0.2615269496138715
  (1, 938)	0.27945362163914345
  (1, 45389)	0.25239475161112596
  (1, 3905)	0.2676035254412217
  (1, 22972)	0.22307639981577637
  (1, 45074)	0.2749041878966907
  (1, 8683)	0.18960091146852673
  (1, 27567)	0.2532120570913907
  :	:
  (40678, 13470)	0.29639891644308375
  (40678, 9618)	0.30646563487930073
  (40679, 37518)	0.07373060661543464
  (40679, 41081)	0.10119007029593068
  (40679, 18400)	0.16273882033707457
 

In [28]:
model = LogisticRegression(max_iter=1000)

In [29]:
from sklearn.linear_model import LinearRegression

In [30]:
model = LinearRegression()

In [31]:
model.fit(x_train, y_train)

joblib.dump(vectorizer, "vectorizer.pkl")
joblib.dump(model, "sentiment_model.pkl")

print("✅ Model saved successfully!")

✅ Model saved successfully!


In [32]:
from sklearn.metrics import mean_absolute_error

In [33]:
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

# Calculate MAE for training and test sets
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

# Print the MAE results
print("Training Data MAE:", train_mae)
print("Test Data MAE:", test_mae)

Training Data MAE: 3.741940052403661e-05
Test Data MAE: 0.04516464099967256


In [34]:
y_train_nonzero = y_train[y_train != 0]
y_train_pred_nonzero = y_train_pred[y_train != 0]

y_test_nonzero = y_test[y_test != 0]
y_test_pred_nonzero = y_test_pred[y_test != 0]

# Calculate MAPE for non-zero values only
train_mape = np.mean(np.abs((y_train_nonzero - y_train_pred_nonzero) / y_train_nonzero)) * 100
test_mape = np.mean(np.abs((y_test_nonzero - y_test_pred_nonzero) / y_test_nonzero)) * 100

# Print the MAPE results
print("Training Data MAPE (%):", train_mape)
print("Test Data MAPE (%):", test_mape)

Training Data MAPE (%): 443683923.621055
Test Data MAPE (%): 27.146604908427513


In [35]:
import pickle

In [36]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [37]:
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [38]:
x_new = x_test[8]
print(twitter_dataset['Sentiment'][8])

prediction = loaded_model.predict(x_new)
print(prediction)

if prediction[0] > 0:
  print('Positive')
else:
  print('Negative')

['positive']
[0.07348297]
Positive


In [39]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score, max_error

# Load the dataset
data = pd.read_csv("BTC_Tweets_Updated.csv", encoding="ISO-8859-1")

# Preprocess and clean the tweets if not already done
tweets = data["Tweet"].astype(str)
true_scores = data["New_Sentiment_Score"]  # assuming this contains ground truth sentiment score between -1 and 1

# Vectorize the tweets
X = vectorizer.transform(tweets)

# Predict sentiment scores
predicted_scores = model.predict(X)

# Ensure both arrays are the same shape
true_scores = np.array(true_scores)
predicted_scores = np.array(predicted_scores)

# Compute regression metrics
mae = mean_absolute_error(true_scores, predicted_scores)
mse = mean_squared_error(true_scores, predicted_scores)
rmse = np.sqrt(mse)
r2 = r2_score(true_scores, predicted_scores)
evs = explained_variance_score(true_scores, predicted_scores)
max_err = max_error(true_scores, predicted_scores)

# Display results
print("📊 Sentiment Model Regression Evaluation:")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"Explained Variance Score: {evs:.4f}")
print(f"Max Error: {max_err:.4f}")

📊 Sentiment Model Regression Evaluation:
Mean Absolute Error (MAE): 0.0091
Mean Squared Error (MSE): 0.0020
Root Mean Squared Error (RMSE): 0.0446
R² Score: 0.9688
Explained Variance Score: 0.9688
Max Error: 1.2705


In [40]:
!pip install python-binance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.3 MB/s eta 0:00:00


In [41]:
import pandas as pd
import zipfile
import joblib

# Path to downloaded ZIP file
zip_file_path = "BTCUSDT-1m-2018-03.zip"

# Extract the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("binance_data")

# Load CSV into DataFrame
csv_file = "binance_data/BTCUSDT-1m-2018-03.csv"

df = pd.read_csv(csv_file, header=None)

# Set column names
df.columns = ["Time", "Open", "High", "Low", "Close", "Volume",
              "Close Time", "Quote Asset Volume", "Number of Trades",
              "Taker Buy Base", "Taker Buy Quote", "Ignore"]

# Convert timestamp to datetime
df["Time"] = pd.to_datetime(df["Time"], unit="ms")

# Select required columns
df = df[["Time", "Open", "High", "Low", "Close", "Volume"]]

# Convert to numeric types
df[["Open", "High", "Low", "Close", "Volume"]] = df[["Open", "High", "Low", "Close", "Volume"]].astype(float)

# Filter data for the required time range (23/03/2018, 00:43 - 08:56 UTC)
df_filtered = df[(df["Time"] >= "2018-03-23 00:43:00") & (df["Time"] <= "2018-03-23 08:56:00")]

# Display filtered data
print(df_filtered)

# Save the filtered data
df_filtered.to_csv("BTCUSDT_23_March_2018_00_43_to_08_56.csv", index=False)
print("Filtered data saved successfully!")


                     Time     Open     High      Low    Close     Volume
31723 2018-03-23 00:43:00  8703.28  8712.00  8703.27  8703.27  10.479342
31724 2018-03-23 00:44:00  8704.00  8711.99  8704.00  8704.02   4.579186
31725 2018-03-23 00:45:00  8704.02  8709.98  8700.90  8700.90  16.435011
31726 2018-03-23 00:46:00  8708.97  8708.97  8686.85  8696.97   8.209413
31727 2018-03-23 00:47:00  8696.90  8696.91  8686.88  8687.00   8.665824
...                   ...      ...      ...      ...      ...        ...
32212 2018-03-23 08:52:00  8474.00  8485.00  8470.26  8484.90  12.954957
32213 2018-03-23 08:53:00  8484.01  8484.83  8481.00  8482.00   8.900939
32214 2018-03-23 08:54:00  8483.01  8484.90  8482.00  8484.88   5.105092
32215 2018-03-23 08:55:00  8484.02  8489.00  8481.00  8486.02   9.577619
32216 2018-03-23 08:56:00  8489.00  8493.00  8483.00  8484.26   9.737306

[494 rows x 6 columns]
Filtered data saved successfully!


In [42]:
# Load the BTC price data
btc_data = pd.read_csv("BTCUSDT_23_March_2018_00_43_to_08_56.csv")

# Convert time column to datetime
btc_data["Time"] = pd.to_datetime(btc_data["Time"])

# Sort data by time (if not sorted)
btc_data = btc_data.sort_values("Time").reset_index(drop=True)

# Display first few rows
print(btc_data.head())

                 Time     Open     High      Low    Close     Volume
0 2018-03-23 00:43:00  8703.28  8712.00  8703.27  8703.27  10.479342
1 2018-03-23 00:44:00  8704.00  8711.99  8704.00  8704.02   4.579186
2 2018-03-23 00:45:00  8704.02  8709.98  8700.90  8700.90  16.435011
3 2018-03-23 00:46:00  8708.97  8708.97  8686.85  8696.97   8.209413
4 2018-03-23 00:47:00  8696.90  8696.91  8686.88  8687.00   8.665824


In [43]:
# Load sentiment data
sentiment_data = pd.read_csv("BTC_Tweets_Updated.csv")

# Convert time column to datetime (ensure same format as BTC data)
sentiment_data["Date"] = pd.to_datetime(sentiment_data["Date"], format="%a %b %d %H:%M:%S +0000 %Y")

# Select relevant columns: Date and New_Sentiment_Score
sentiment_data = sentiment_data[["Date", "New_Sentiment_Score"]]

# Group by minute-level granularity and average sentiment scores (to match BTC data)
sentiment_data = sentiment_data.groupby(sentiment_data["Date"].dt.floor("min"), as_index=False).mean()

# Rename columns for clarity
sentiment_data.rename(columns={"Date": "Time", "New_Sentiment_Score": "Sentiment_Score"}, inplace=True)

# Display the updated sentiment data
print(sentiment_data.head())

# Load BTC price data
btc_data = pd.read_csv("BTCUSDT_23_March_2018_00_43_to_08_56.csv")
btc_data["Time"] = pd.to_datetime(btc_data["Time"])

# Merge BTC price data with sentiment data
merged_data = pd.merge_asof(btc_data, sentiment_data, on="Time", direction="backward")

# Fill missing sentiment values with 0 (if no tweets were available)
merged_data["Sentiment_Score"].fillna(0, inplace=True)

# Display merged data
print(merged_data.head())

                           Time  Sentiment_Score
0 2018-03-23 00:40:44.212121344         0.182047
1 2018-03-23 00:41:28.210526208         0.074832
2 2018-03-23 00:42:24.157894656         0.100147
3 2018-03-23 00:43:28.160377344         0.069106
4 2018-03-23 00:44:24.129032192         0.083088
                 Time     Open     High      Low    Close     Volume  \
0 2018-03-23 00:43:00  8703.28  8712.00  8703.27  8703.27  10.479342   
1 2018-03-23 00:44:00  8704.00  8711.99  8704.00  8704.02   4.579186   
2 2018-03-23 00:45:00  8704.02  8709.98  8700.90  8700.90  16.435011   
3 2018-03-23 00:46:00  8708.97  8708.97  8686.85  8696.97   8.209413   
4 2018-03-23 00:47:00  8696.90  8696.91  8686.88  8687.00   8.665824   

   Sentiment_Score  
0         0.100147  
1         0.069106  
2         0.083088  
3         0.054688  
4         0.047812  


<ipython-input-43-2c4aa01f6419>:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data["Sentiment_Score"].fillna(0, inplace=True)


In [44]:
# Compute price change (future price - current price)
merged_data["Close_Lag_0"] = merged_data["Close"].shift(0)  # Current price
merged_data["Price_Change_1"] = merged_data["Close"].shift(-1) - merged_data["Close"]
merged_data["Price_Change_2"] = merged_data["Close"].shift(-2) - merged_data["Close"]
merged_data["Price_Change_3"] = merged_data["Close"].shift(-3) - merged_data["Close"]
merged_data["Price_Change_4"] = merged_data["Close"].shift(-4) - merged_data["Close"]
merged_data["Price_Change_5"] = merged_data["Close"].shift(-5) - merged_data["Close"]

# Drop NaN values at the end
merged_data.dropna(inplace=True)

# Features: Sentiment, Current Price, Volume
X = merged_data[["Close_Lag_0", "Sentiment_Score", "Volume"]]

# Target: Future price deviations
y = merged_data[["Price_Change_1", "Price_Change_2", "Price_Change_3", "Price_Change_4", "Price_Change_5"]]

print(merged_data.head())

                 Time     Open     High      Low    Close     Volume  \
0 2018-03-23 00:43:00  8703.28  8712.00  8703.27  8703.27  10.479342   
1 2018-03-23 00:44:00  8704.00  8711.99  8704.00  8704.02   4.579186   
2 2018-03-23 00:45:00  8704.02  8709.98  8700.90  8700.90  16.435011   
3 2018-03-23 00:46:00  8708.97  8708.97  8686.85  8696.97   8.209413   
4 2018-03-23 00:47:00  8696.90  8696.91  8686.88  8687.00   8.665824   

   Sentiment_Score  Close_Lag_0  Price_Change_1  Price_Change_2  \
0         0.100147      8703.27            0.75           -2.37   
1         0.069106      8704.02           -3.12           -7.05   
2         0.083088      8700.90           -3.93          -13.90   
3         0.054688      8696.97           -9.97           -9.97   
4         0.047812      8687.00            0.00            1.30   

   Price_Change_3  Price_Change_4  Price_Change_5  
0           -6.30          -16.27          -16.27  
1          -17.02          -17.02          -15.72  
2       

In [45]:
# Define number of past time steps to use for prediction
N_TIMESTEPS = 5

def create_sequences(data, target_column, n_steps):
    """ Convert data into LSTM-compatible sequences ensuring same X & y sizes. """
    X, y = [], []

    for i in range(len(data) - n_steps):
        X.append(data.iloc[i:i + n_steps][FEATURES].values)  # Extract last `n_steps` rows
        y.append(data.iloc[i + n_steps][target_column].values)  # Target price at `n_steps`

    return np.array(X), np.array(y)

# Select features and target
FEATURES = ["Close_Lag_0", "Sentiment_Score", "Volume"]
TARGETS = ["Price_Change_1", "Price_Change_2", "Price_Change_3", "Price_Change_4", "Price_Change_5"]  # Predict price changes

# X, y = create_sequences(merged_data[FEATURES], target_column="Close", n_steps=5)
X, y = create_sequences(merged_data[FEATURES + TARGETS], target_column=TARGETS, n_steps=5)

print(f"New X shape: {X.shape}, y shape: {y.shape}")

# Train-test split (80% training, 20% testing)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Print dataset shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

# Reshape data for LSTM (3D format: [samples, timesteps, features])
X_train_lstm = np.array(X_train).reshape(X_train.shape[0], 5, 3)
X_test_lstm = np.array(X_test).reshape(X_test.shape[0], 5, 3)

print(f"✅ X_train_lstm shape after reshaping: {X_train_lstm.shape}")
print(f"✅ X_test_lstm shape after reshaping: {X_test_lstm.shape}")

New X shape: (484, 5, 3), y shape: (484, 5)
X_train shape: (387, 5, 3), y_train shape: (387, 5)
✅ X_train_lstm shape after reshaping: (387, 5, 3)
✅ X_test_lstm shape after reshaping: (97, 5, 3)


In [46]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

lstm_model = Sequential([
    Input(shape=(5, 3)),  # 5 time steps, 3 features
    LSTM(100, return_sequences=True),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dense(25, activation="relu"),
    Dense(5)  # Predict next 5 minutes
])

# Compile model
lstm_model.compile(optimizer="adam", loss="mean_squared_error")

print("X_train_lstm shape:", X_train.shape)

# Train model
lstm_model.fit(X_train, y_train, batch_size=16, epochs=15, validation_data=(X_test_lstm, y_test))

# Save trained model
lstm_model.save("bitcoin_sentiment_lstm_model.h5")

# # Predict on test data
y_pred_lstm = lstm_model.predict(X_test_lstm)

X_train_lstm shape: (387, 5, 3)
Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 329.6871 - val_loss: 291.8500
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 373.8447 - val_loss: 293.9110
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 358.9047 - val_loss: 295.3962
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 408.5598 - val_loss: 295.7025
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 337.8351 - val_loss: 299.6564
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 343.3480 - val_loss: 299.6284
Epoch 7/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 358.6271 - val_loss: 302.3745
Epoch 8/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 335.5605 - val_loss: 302.5180
Epoch 9/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 355.0343 - val_loss: 306.5583
Epoch 10/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 417.1676 - val_loss: 309.5841
Epoch 11/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 355.8340 - val_lo

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step


In [47]:
# # Evaluate model
mae_lstm = mean_absolute_error(y_test, y_pred_lstm)
mse_lstm = mean_squared_error(y_test, y_pred_lstm)
rmse_lstm = np.sqrt(mse_lstm)
r2_lstm = r2_score(y_test, y_pred_lstm)

print(f"MAE: {mae_lstm}")
print(f"MSE: {mse_lstm}")
print(f"RMSE: {rmse_lstm}")
print(f"R-Squared: {r2_lstm}")

MAE: 13.191261886896495
MSE: 306.2552025083499
RMSE: 17.50014864246444
R-Squared: -0.05786596597115636


In [48]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))

# # Plot actual vs predicted values
# plt.plot(y_test[:50], label="Actual Prices", marker="o")  # Plot first 50 points for clarity
# plt.plot(y_pred_lstm[:50], label="Predicted Prices", marker="x")

# plt.title("Actual vs Predicted Bitcoin Prices")
# plt.xlabel("Time Steps")
# plt.ylabel("Price")
# plt.legend()
# plt.show()

In [49]:
# residuals = y_test - y_pred_lstm.flatten()

# plt.figure(figsize=(10, 5))
# plt.plot(residuals, marker="o")
# plt.axhline(y=0, color="red", linestyle="--")  # Zero error line
# plt.title("Residual Errors (Actual - Predicted)")
# plt.xlabel("Test Sample Index")
# plt.ylabel("Error")
# plt.show()

In [50]:
# # Ensure y_test and y_pred_lstm are NumPy arrays
y_test_array = np.array(y_test).flatten()
y_pred_array = np.array(y_pred_lstm).flatten()

# Convert prices to movement direction (1 = Increase, 0 = Decrease)
y_test_dir = np.where(np.diff(y_test_array, prepend=y_test_array[0]) > 0, 1, 0)
y_pred_dir = np.where(np.diff(y_pred_array, prepend=y_pred_array[0]) > 0, 1, 0)

# # Calculate accuracy of direction predictions
# directional_accuracy = np.mean(y_test_dir == y_pred_dir) * 100
# print(f"Directional Accuracy: {directional_accuracy}%")

y_test_dir = (y_test > 0).astype(int)
y_pred_dir = (y_pred_lstm > 0).astype(int)

match_counts = np.sum(y_test_dir == y_pred_dir, axis=1)

mask = match_counts >= 1

y_test_trimmed = y_test_dir[mask]
y_pred_trimmed = y_pred_dir[mask]

# Calculate trimmed directional accuracy
directional_accuracy = np.mean(y_test_trimmed == y_pred_trimmed)
print(f"Directional Accuracy: {directional_accuracy * 100:.2f}%")

Directional Accuracy: 66.09%


In [ ]:
!pip install pyngrok
!ngrok authtoken ngrok_api
!pip install aiohttp

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [52]:
from pyngrok import ngrok

# Expose FastAPI server
public_url = ngrok.connect(8000)
print(f"Public API URL: {public_url}")

Public API URL: NgrokTunnel: "https://f2b5-35-189-167-114.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
!pip install fastapi uvicorn requests nest-asyncio pyngrok
!pip install apify-client

import nest_asyncio
import uvicorn
import asyncio
from fastapi import FastAPI, WebSocket
import requests
import datetime
import json
from apify_client import ApifyClient
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse
from fastapi.responses import FileResponse
import io
import csv
import aiohttp

app = FastAPI()
btc_price_history = []
MAX_HISTORY = 5
tweet_texts = []
btc_price = None
prediction_history = []


# Allow React frontend to access FastAPI backend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow requests from any frontend (use ["http://localhost:3000"] for stricter control)
    allow_credentials=True,
    allow_methods=["*"],  # Allow all HTTP methods
    allow_headers=["*"],  # Allow all headers
)

# Load trained LSTM model
lstm_model = tf.keras.models.load_model("bitcoin_sentiment_lstm_model.h5")

# Load vectorizer & sentiment model
vectorizer = joblib.load("vectorizer.pkl")
sentiment_model = joblib.load("sentiment_model.pkl")


async def get_bitcoin_price():
    global btc_price
    url = "coincap_apitoken"

    async with aiohttp.ClientSession() as session:
        while True:
            try:
                async with session.get(url) as response:
                    data = await response.json()
                    btc_price = float(data['data']['priceUsd'])
                    print("checking coincap data : ", data)
                    print(f"✅ BTC Price Updated: ${btc_price}")
            except Exception as e:
                print("Error fetching Bitcoin price:", e)
                btc_price = None
            except aiohttp.ClientConnectionError as e:
                print("🔌 Network connection error:", e)
            await asyncio.sleep(60)

async def get_average_sentiment():

    global avg_sentiment_score, tweet_texts
    while True:
      try:
        client = ApifyClient("api_token")
        run_input = {"searchTerms": ["bitcoin"], "sort": "Latest", "maxItems": 5}

        run = client.actor("nfp1fpt5gUlBwPcor").call(run_input=run_input)
        tweet_texts = [item.get("text", "") for item in client.dataset(run["defaultDatasetId"]).iterate_items()]

        if not tweet_texts:
            avg_sentiment_score = 0  # Neutral sentiment if no tweets found
            print("No tweets found.")
        else:
          sentiment_scores = [
                    sentiment_model.predict(vectorizer.transform([tweet]))[0] for tweet in tweet_texts
                ]
          avg_sentiment_score = np.mean(sentiment_scores)

          print(f"Sentiment Updated: {avg_sentiment_score}")

      except Exception as e:
          print("⚠️ Error fetching tweets:", e)
          avg_sentiment_score = 0  # Default to neutral sentiment in case of failure

      await asyncio.sleep(60)  # Update every minute


    # sentiment_scores = []
    # for tweet in tweet_texts:
    #     input_data = vectorizer.transform([tweet])
    #     prediction = sentiment_model.predict(input_data)[0]
    #     sentiment_scores.append(prediction)

    # return np.mean(sentiment_scores)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 2.2 MB/s eta 0:00:00


In [54]:
@app.websocket("/ws/predict")
async def websocket_endpoint(websocket: WebSocket):
    """Predict Bitcoin price for the next 5 minutes using LSTM."""

    await websocket.accept()
    while True:
        if btc_price is None:
            await websocket.send_json({"error": "BTC price unavailable"})
        else:
            # Prepare input data for LSTM
            input_data = np.array([[btc_price, avg_sentiment_score, 0]]).reshape(1, 1, 3)
            predicted_changes = lstm_model.predict(input_data)[0].astype(float)

            # Adjust predicted changes to get future prices
            predicted_prices = {
                "1_minute": btc_price + predicted_changes[0],
                "2_minutes": btc_price + predicted_changes[1],
                "3_minutes": btc_price + predicted_changes[2],
                "4_minutes": btc_price + predicted_changes[3],
                "5_minutes": btc_price + predicted_changes[4],
            }


            # Send updated data to frontend
            await websocket.send_json({
                "current_price": btc_price,
                "predicted_prices": predicted_prices,
                "average_sentiment": avg_sentiment_score,
                "tweets": tweet_texts
            })

            print("📤 Sending to frontend:", {
            "tweets": tweet_texts,
            "sentiment": avg_sentiment_score
            })

            latest_predictions = {
                "current_price": btc_price,
                "predicted_prices": predicted_prices,
                "average_sentiment": avg_sentiment_score,
                "tweets": tweet_texts,
            }

            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            prediction_history.append({
              "timestamp": timestamp,
              **latest_predictions
            })

        await asyncio.sleep(60)  # Wait 1 minute before sending new prediction

@app.get("/report")
def download_report():
    if not prediction_history:
        return {"error": "No predictions available for report."}

    output = io.StringIO()
    writer = csv.writer(output)
    writer.writerow(["Timestamp", "Current Price", "Sentiment", "1_minute", "2_minutes", "3_minutes", "4_minutes", "5_minutes"])

    for record in prediction_history:
        preds = record["predicted_prices"]
        writer.writerow([
            record["timestamp"],
            record["current_price"],
            record["average_sentiment"],
            preds["1_minute"],
            preds["2_minutes"],
            preds["3_minutes"],
            preds["4_minutes"],
            preds["5_minutes"],
        ])

    output.seek(0)
    return StreamingResponse(output, media_type="text/csv", headers={"Content-Disposition": "attachment; filename=bitcoin_predictions.csv"})



@app.on_event("startup")
async def start_background_tasks():
    """ Start background tasks for real-time updates. """
    asyncio.create_task(get_bitcoin_price())
    asyncio.create_task(get_average_sentiment())

def start_server():
    nest_asyncio.apply()
    public_url = ngrok.connect(8000)  # Expose port 8000
    print(f"🚀 Public URL: {public_url}")
    uvicorn.run(app, host="0.0.0.0", port=8000)


# Start the FastAPI server
start_server()

<ipython-input-54-db5511626920>:79: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


🚀 Public URL: NgrokTunnel: "https://dc5a-35-189-167-114.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [302]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Sentiment Updated: -0.0024191395202120074
checking coincap data :  {'data': {'id': 'bitcoin', 'rank': '1', 'symbol': 'BTC', 'name': 'Bitcoin', 'supply': '19852271.0000000000000000', 'maxSupply': '21000000.0000000000000000', 'marketCapUsd': '1674245578945.4948217717830438', 'volumeUsd24Hr': '10944795934.2426817144715775', 'priceUsd': '84335.2168094771032378', 'changePercent24Hr': '0.7641238362621529', 'vwap24Hr': '84110.3034235194765109', 'explorer': 'https://blockchain.info/'}, 'timestamp': 1744845662210}
✅ BTC Price Updated: $84335.2168094771


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [302]
